In [2]:
import pandas as pd
import numpy as np

In [26]:
train = pd.read_csv('../data/train_1M.csv')
questions = pd.read_csv('../data/questions.csv')

In [27]:
# Only keep questions, rename content_id and merge with questions table.
train = train.query('content_type_id==0')
train = train.rename(columns={'content_id': 'question_id'})
train = train.merge(questions, on='question_id', how='left')
# Add features.
train['user_activity_cumcount']=train.groupby('user_id').cumcount()
train['question_avg_score'] = train.groupby('question_id')['answered_correctly'].mean()
train['avgscore_cummean'] = train.groupby('user_id').answered_correctly.apply(lambda x: x.shift().expanding().mean())
# Then remove all columns but 'question_avg_score', 'user_avgscore_cummean', 'answered_correctly' (target). Replace inf by 0
b0_train = train[['question_avg_score', 'avgscore_cummean', 'answered_correctly']]
b0_train.avgscore_cummean.replace(np.nan, 0, inplace=True)

/Users/Yohann/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [28]:
#b0_train.to_csv('b0_model.csv')

In [29]:
b0_train.head(50)

,question_avg_score,avgscore_cummean,answered_correctly
0,0.886076,0.000000,1
1,0.842857,1.000000,0
2,0.544355,0.500000,0
3,0.741935,0.333333,0
4,0.612378,0.250000,0
5,0.865169,0.200000,1
6,0.508079,0.333333,0
7,0.841176,0.285714,0
8,0.910112,0.250000,1
9,0.306613,0.333333,0


In [45]:
#import matplotlib.pyplot as plt
#plt.scatter(b0_train[0,:], b0_train[1,:])

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn. metrics import roc_auc_score, accuracy_score


log = LogisticRegression(solver='saga', max_iter=1000)

X = b0_train.drop(columns=['question_avg_score', 'avgscore_cummean'])
y = b0_train.answered_correctly
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 1)

In [37]:
y_train.shape

(677483,)

In [40]:
log.fit(X_train, y_train)
y_pred = log.predict(X_test)

#cv_results = cross_validate(log, X, y, scoring='accuracy')

#print(cv_results['test_score'].mean())


#y_pred = cross_val_predict(log, X, y, cv=10)
accuracy = accuracy_score(y_test, y_pred)


In [41]:
accuracy

1.0


In [22]:
b0_train.answered_correctly.value_counts()

1    635571
0    332263
Name: answered_correctly, dtype: int64

In [24]:
y_pred.nunique()

AttributeError: 'numpy.ndarray' object has no attribute 'nunique'

In [16]:
import sklearn.metrics
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']